## **Using Pandas library to Analyze a videogames Dataset** 🎮


In [ ]:
import pandas as pd 
import numpy as np 
import sqlite3 
import matplotlib as plt

In [ ]:
videogames = pd.read_csv('dato.csv', sep = ',')

### Visualização inicial do dataframe

In [ ]:
videogames.head()

,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Rating,Critic_Score_Class
0,Wii,Sports,Nintendo,"41,36","28,96","3,77","8,45","82,54",E,Bueno
1,Wii,Racing,Nintendo,"15,68","12,8","3,79","3,29","35,57",E,Excelente
2,Wii,Sports,Nintendo,"15,61","10,95","3,28","2,95","32,78",E,Excelente
3,DS,Platform,Nintendo,"11,28","9,15","6,5","2,88","29,81",E,Excelente
4,Wii,Misc,Nintendo,"13,96","9,18","2,93","2,84","28,92",E,Malo


### Tradução das classificações dos jogos do espanhol para português

In [ ]:
for idx, row in videogames.iterrows(): 
  if row['Critic_Score_Class'] == 'Bueno':  
    row['Critic_Score_Class'] = 'BOM'
  
  elif row['Critic_Score_Class'] == 'Malo': 
    row['Critic_Score_Class']  =  'RUIM' 

  elif row['Critic_Score_Class'] =='Excelente': 
    row['Critic_Score_Class'] = 'EXCELENTE'
  

### Aquireordenamos as colunas do nosso dataframe 'videogames' para termos dados categóricos no início e dados quantitativos referentes às vendas no final, apenas para fins didáticos. 

In [ ]:
videogames = videogames.reindex(columns=['Platform','Genre','Publisher','Critic_Score_Class','Rating','NA_Sales',\
'EU_Sales','JP_Sales','Other_Sales','Global_Sales'])

In [ ]:
videogames.head()

,Platform,Genre,Publisher,Critic_Score_Class,Rating,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii,Sports,Nintendo,BOM,E,"41,36","28,96","3,77","8,45","82,54"
1,Wii,Racing,Nintendo,EXCELENTE,E,"15,68","12,8","3,79","3,29","35,57"
2,Wii,Sports,Nintendo,EXCELENTE,E,"15,61","10,95","3,28","2,95","32,78"
3,DS,Platform,Nintendo,EXCELENTE,E,"11,28","9,15","6,5","2,88","29,81"
4,Wii,Misc,Nintendo,RUIM,E,"13,96","9,18","2,93","2,84","28,92"



### Utilizando o Sqlite3 para facilitar as consultas mediante uso do SQL junto ao pandas.

In [ ]:
connection = sqlite3.connect('videogames') 
videogames.to_sql('videogames', connection, if_exists='replace', index=False)

7112

### Após conectarmos o sqlite3 ao dataframe pandas e convertê-lo ao formato sql, podemos realizar consultas no padrão SQL no dataframe, o que facilita a obtenção de dados e visualização de padrões. 
### Na consulta abaixo estamo selecionando os gêneros de jogos classificados como bons, de maneira única e ordenando pela quantidade de vendas no Japão, América do Norte e Europa, consecutivamente.

In [ ]:
result_japan = pd.read_sql_query("SELECT DISTINCT Genre FROM videogames WHERE Critic_Score_Class = 'BOM' ORDER BY JP_Sales", connection) 
result_north_america = pd.read_sql_query("SELECT DISTINCT Genre FROM videogames WHERE Critic_Score_Class = 'BOM' ORDER BY NA_Sales", connection) 
result_europe = pd.read_sql_query("SELECT DISTINCT Genre FROM videogames WHERE Critic_Score_Class = 'BOM' ORDER BY EU_sales", connection) 

### Avaliando as plataformas de acordo com a quantidade de jogos em cada categoria de classificação (EXCELENTE, BOM ou RUIM)

In [ ]:
evaluation_platform_goodgames = pd.read_sql_query("SELECT DISTINCT Platform, COUNT(Critic_Score_Class) FROM videogames WHERE Critic_Score_Class = 'BOM' GROUP BY Platform ORDER BY COUNT(Critic_Score_Class) DESC", connection)
evaluation_platform_excgames = pd.read_sql_query("SELECT DISTINCT Platform, COUNT(Critic_Score_Class) FROM videogames WHERE Critic_Score_Class = 'EXCELENTE' GROUP BY Platform ORDER BY COUNT(Critic_Score_Class) DESC", connection)
evaluation_platform_badgames = pd.read_sql_query("SELECT DISTINCT Platform, COUNT(Critic_Score_Class) FROM videogames WHERE Critic_Score_Class = 'RUIM' GROUP BY Platform ORDER BY COUNT(Critic_Score_Class) DESC", connection)

### Avaliando a quantidade de vendas globais por classificação

In [ ]:
sales_goodgames = pd.read_sql_query("SELECT DISTINCT Critic_Score_Class, SUM(Global_Sales) FROM videogames WHERE Critic_Score_Class = 'BOM' ",connection)
sales_excgames = pd.read_sql_query("SELECT DISTINCT Critic_Score_Class, SUM(Global_Sales) FROM videogames WHERE Critic_Score_Class = 'EXCELENTE' ",connection)
sales_badgames = pd.read_sql_query("SELECT DISTINCT Critic_Score_Class, SUM(Global_Sales) FROM videogames WHERE Critic_Score_Class = 'RUIM' ",connection)

In [ ]:
sales_by_evaluation = pd.DataFrame(sales_goodgames) 
sales_by_evaluation = sales_by_evaluation.append(sales_excgames)
sales_by_evaluation = sales_by_evaluation.append(sales_badgames)


In [ ]:
sales_by_evaluation = sales_by_evaluation.sort_values(by=['SUM(Global_Sales)'], ascending = False) 
sales_by_evaluation

,Critic_Score_Class,SUM(Global_Sales)
0,EXCELENTE,2220.0
0,BOM,660.0
0,RUIM,119.0
